In [46]:
!pip install kaggle

Importing the Dependencies

In [47]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
#Sequential layer of neural network
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#Dense -> fully connected layers
#Embedding -> First layer in LSTM
#LSTM layer

In [48]:
#Load the tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#mapping the words to integer or vectors
#input data of the same shape

DATA COLLECTION THROUGH API

In [49]:
kaggle_dictionary = json.load(open("kaggle.json"))
#Json object to python

In [50]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [51]:
#setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [52]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [53]:
ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data/


In [54]:
# unzip the dataset file read the file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

Load the dataset

In [55]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [56]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [57]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [58]:
data.shape

(50000, 2)

In [59]:
data["sentiment"].value_counts()
#balanced data

,count
sentiment,
positive,25000
negative,25000


In [60]:
#Encoding
data.replace({"sentiment":{"positive":1 , "negative":0}},inplace=True)

<ipython-input-60-ab122c3a9a89>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1 , "negative":0}},inplace=True)


In [61]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [63]:
train_data,test_data = train_test_split(data,test_size=0.2, random_state=42)

In [64]:
train_data.shape

(40000, 2)

In [65]:
test_data.shape

(10000, 2)

DATA PREPROCESSING

In [66]:
#Tokenizing text data
tokenizer = Tokenizer(num_words=5000)
#conversion of words into numbers
#most common 5000 words will be converted to number

In [67]:
tokenizer.fit_on_texts(train_data["review"])

In [86]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
#same length
#Conversion to sequence of numbers
X_test= pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [87]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [88]:
X_train.shape

(40000, 200)

In [89]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [90]:
X_test.shape

(10000, 200)

In [91]:
Y_train = train_data["sentiment"]

In [92]:
Y_test =  test_data["sentiment"]

In [93]:
Y_test

,sentiment
33553,1
9427,1
199,0
12447,1
39489,0
...,...
28567,0
25079,1
18707,1
15200,0


Building the LSTM model

In [94]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
#dropout -> avoid overfitting



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [95]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [96]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

TRAIN THE MODEL

In [ ]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 245s 483ms/step - accuracy: 0.7034 - loss: 0.5499 - val_accuracy: 0.7912 - val_loss: 0.4453
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 261s 483ms/step - accuracy: 0.8356 - loss: 0.3810 - val_accuracy: 0.8515 - val_loss: 0.3545
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 261s 482ms/step - accuracy: 0.8659 - loss: 0.3278 - val_accuracy: 0.8639 - val_loss: 0.3285
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 260s 478ms/step - accuracy: 0.8832 - loss: 0.2886 - val_accuracy: 0.8625 - val_loss: 0.3275
Epoch 5/5
169/500 ━━━━━━━━━━━━━━━━━━━━ 2:25 439ms/step - accuracy: 0.9017 - loss: 0.2480

MODEL EVALUATION

In [98]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 122ms/step - accuracy: 0.8710 - loss: 0.3158
Test Loss: 0.3123248219490051
Test Accuracy: 0.8730000257492065


PREDICTIVE SYSTEM

In [99]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [100]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
The sentiment of the review is: positive
